In [4]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')
!pip install -r requirements.txt

import PETs_Experiment

  Obtaining dependency information for pip>=23.3 from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 435.7 kB/s eta 0:00:05
    --------------------------------------- 0.0/2.1 MB 435.7 kB/s eta 0:00:05
   - -------------------------------------- 0.1/2.1 MB 409.6 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/2.1 MB 595.3 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/2.1 MB 595.3 kB/s eta 0:00:04
   ---- ----------------------------------- 0.2/2.1 MB 765.3 kB/s eta 0:00:03
   ---- ----------------------------------- 0.2/2.1 MB 765.3 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.1 MB 741.6 kB/s eta 0:00:03
   ------- -----------------------------

ERROR: To modify pip, please run the following command:
C:\Program Files\Python312\python.exe -m pip install -r requirements.txt

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Program Files\Python312\python.exe -m pip install --upgrade pip


In [2]:
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
                               ,na_values = {k : '?' for k in ['workclass'
                                                              ,'occupation'
                                                              ,'native-country'
                                                              ]
                                            }
                               )
print(loader.data.head(1))

   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [3]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

39073
9769
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  
   age workclass  fnlwgt education  educational-num      marital-status  \
0   27   Private  205145   HS-grad                9  Married-civ-spouse   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct      Husband  White   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [4]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        ,outlier_columns = {'ignore': ['hours-per-week']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        ,scaling_method  = 'Standard'
        ,scaling_columns = {'focus' : 'fnlwgt'}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Missingist (Drop): Dropped 2895 rows.
Preprocessor - Outlierist (IQR): Dropped  3013 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   233 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   225 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1727 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1059 rows on fnlwgt         . Kept [-63222.875, 418198.125] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6047 in 36178 rows.
Preprocessor - Encoder (Label): Column relationship    been labelized from 0 to  5.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column income          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column marital-status 

In [5]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0254 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.0482 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30131 rows (same as raw) in 1.6538 sec.
   age  workclass    fnlwgt  education  educational-num  marital-status  \
0   43          2 -0.495437         13               10               0   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0           4             1     4       0             0             0   

   hours-per-week  native-country  income  
0              43              24       1  


In [6]:
postproc = PETs_Experiment.Postprocessor(data = synthesizer.data_syn
                                        ,encoder = getattr(preproc ,'encoder' ,None)
                                        ,scaler  = getattr(preproc ,'scaler'  ,None)
                                        )
print(postproc.data.head(1))

Postprocessor - Scaler (StandardScaler): Decoding fnlwgt.
Postprocessor - Encoder (LabelEncoder): Decoding relationship.
Postprocessor - Encoder (LabelEncoder): Decoding workclass.
Postprocessor - Encoder (LabelEncoder): Decoding education.
Postprocessor - Encoder (LabelEncoder): Decoding income.
Postprocessor - Encoder (LabelEncoder): Decoding marital-status.
Postprocessor - Encoder (LabelEncoder): Decoding gender.
Postprocessor - Encoder (LabelEncoder): Decoding native-country.
Postprocessor - Encoder (LabelEncoder): Decoding occupation.
Postprocessor - Encoder (LabelEncoder): Decoding race.
   age workclass         fnlwgt     education  educational-num marital-status  \
0   43   Private  136087.586752  Some-college               10       Divorced   

        occupation   relationship   race  gender  capital-gain  capital-loss  \
0  Farming-fishing  Not-in-family  White  Female             0             0   

   hours-per-week native-country income  
0              43         Mexico 

In [2]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment


para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 30
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run_single_process()
executor.run()


for k ,v in executor.loader.items():
    print(k)
    print(v.data.head(1))

Executor - Loader: [Adt Income] adult.csv loading time: 9.3513 sec.
Executor - Splitter: Default splitting time: 0.0887 sec.
Preprocessor - Missingist (Drop): Dropped 2921 rows.
Preprocessor - Outlierist (IQR): Dropped   231 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   205 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1082 rows on fnlwgt         . Kept [-62854.25, 418079.75] only.
Preprocessor - Outlierist (IQR): Dropped  1695 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  3021 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6027 in 36152 rows.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (Label): Column education       been labelized from 0 to 13.
Preprocessor - Encoder (Label): Column occupation      been labelized from 0 to 13.
Preprocessor - 

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 15.5713 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30125 rows (same as raw) in 1.5274 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 25.6863 sec.
Preprocessor - Missingist (Drop): Dropped 2893 rows.
Preprocessor - Outlierist (IQR): Dropped   227 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped   227 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1065 rows on fnlwgt         . Kept [-62448.0, 417096.0] only.
Preprocessor - Outlierist (IQR): Dropped  1695 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  3043 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6044 in 36180 rows.
Preprocessor - Encoder (Label): Column race            been labelized from 0 to  4.
Preprocessor - Encoder (Label): Column educat

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.3708 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30136 rows (same as raw) in 1.4446 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 15.8484 sec.
====== ====== ====== ====== ====== ======
Executor: Total execution time: 52.3326 sec.
====== ====== ====== ====== ====== ======


Loading:   0%|          | 0/1 [00:00<?, ?it/s]




Loading: 100%|██████████| 1/1 [00:10<00:00, 10.84s/it]

Loading: 100%|██████████| 1/1 [00:12<00:00, 12.24s/it]


====== ====== ====== ====== ====== ======
Executor: Total execution time: 13.1184 sec.
====== ====== ====== ====== ====== ======
[Adt Income] adult.csv
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment

para_Executor = {
     'Loader' : {
         'adult'  : {'filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
                    ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        }
        ,'NHANES' : {'filepath'     : '../[sunset]/data/[NHANES] B.csv'
                    ,'header_exist' : False
                    ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        }
     }
    ,'Splitter' : {
         '0.8'  : {'num_samples'       : 30
                  ,'train_split_ratio' : 0.8
        }
        ,'0.05' : {'num_samples'       : 30
                  ,'train_split_ratio' : 0.05
        }
    }
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    # ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    # ,'Preprocessor_scaling'         : True
    # ,'Preprocessor_scaling_method'  : 'Standard'
    # ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer': {
        'GaussianCoupula' : {'synthesizing_method' : 'sdv-singletable-gaussiancoupula'}
       ,'CoupulaGAN'      : {'synthesizing_method' : 'sdv-singletable-coupulagan'}
    #    ,'CTGAN'           : {'synthesizing_method' : 'sdv-singletable-ctgan'}
    #    ,'TVAE'            : {'synthesizing_method' : 'sdv-singletable-tvae'}
    }
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()
    

Loading:   0%|          | 0/2 [00:00<?, ?it/s]




Loading:  50%|█████     | 1/2 [00:15<00:15, 15.84s/it]

Loading: 100%|██████████| 2/2 [01:08<00:00, 37.65s/it]

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0236 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.108 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'bmi'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'bmi'. Data will not be rounded.
  warnings.warn(


...

Synthesizing:   0%|          | 1/240 [04:19<17:13:06, 259.36s/it]

Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN spent 27.087 sec.


Synthesizer (SDV - SingleTable): Sampling CoupulaGAN # 109 rows (same as raw) in 0.2554 sec.
Executor - Synthesizer: CoupulaGAN synthesizing time: 53.7021 sec.


Loading: 100%|██████████| 2/2 [04:44<00:00, 142.40s/it]


====== ====== ====== ====== ====== ======
Executor: Total execution time: 285.8287 sec.
====== ====== ====== ====== ====== ======


In [1]:
executor

NameError: name 'executor' is not defined